- Word2Vec

  distributed representation으로 단어의 유사도 계산 가능

  CBOW & Skip-gram

- GloVe

  LSA : 각 단어의 빈도수를 카운트 한 행렬이라는 전체적인 통계 정보를 입력으로 받아 차원을 축소하여 잠재된 의미를 끌어내는 방법론 (의미 유추는 불가)

  word2vec : 실제값과 예측값에 대한 오차를 손실 함수를 통해 줄여가며 학습하는 예측 기반 방법론 (코퍼스의 전체적인 통계 정보는 반영 불가)

  -> GloVe는 두가지를 모두 반영

  : 임베딩 된 중심 단어와 주변 단어 벡터의 내적이 전체 코퍼스에서 Co-occurrence Probability이 되도록 만드는 것

- nn.Embedding() in PyTorch

  단어 -> 단어에 부여된 정수값(not one-hot) -> 임베딩 층 통과 -> 밀집 벡터



In [1]:
# 룩업 테이블 구현

train_data = 'you need to know how to code'

word_set = set(train_data.split())
vocab = {word:i+2 for i,word in enumerate(word_set)}
vocab['<unk>'] = 0
vocab['<pad>'] = 1
vocab

{'<pad>': 1,
 '<unk>': 0,
 'code': 2,
 'how': 7,
 'know': 3,
 'need': 6,
 'to': 5,
 'you': 4}

In [2]:
import torch
embedding_table = torch.FloatTensor([
                               [ 0.0,  0.0,  0.0],
                               [ 0.0,  0.0,  0.0],
                               [ 0.2,  0.9,  0.3],
                               [ 0.1,  0.5,  0.7],
                               [ 0.2,  0.1,  0.8],
                               [ 0.4,  0.1,  0.1],
                               [ 0.1,  0.8,  0.9],
                               [ 0.6,  0.1,  0.1]])

In [3]:
sample = 'you need to run'.split()
idx = []

for w in sample:
  try:
    idx.append(vocab[w])
  except KeyError:
    idx.append(vocab['<unk>'])

idx = torch.LongTensor(idx)

lookup = embedding_table[idx,:]
lookup

tensor([[0.2000, 0.1000, 0.8000],
        [0.1000, 0.8000, 0.9000],
        [0.4000, 0.1000, 0.1000],
        [0.0000, 0.0000, 0.0000]])

In [4]:
# nn.Embedding() 사용
import torch.nn as nn
embedding_layer = nn.Embedding(num_embeddings = len(vocab), # 단어집합 크기
                               embedding_dim = 3,
                               padding_idx = 1)

embedding_layer.weight

Parameter containing:
tensor([[-1.1271,  0.1678,  0.1965],
        [ 0.0000,  0.0000,  0.0000],
        [-0.4390,  1.5269, -0.7928],
        [-0.2572,  0.1868, -0.6623],
        [ 1.2374,  0.7994,  1.0443],
        [ 0.7624, -0.4105,  1.4969],
        [-0.2873,  0.0816, -1.0918],
        [ 2.7678,  0.1585,  0.8112]], requires_grad=True)

- Pretrained Word Embedding



In [5]:
# using IMDB review data for training

from torchtext.legacy import data, datasets

TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True)

trainset, testset = datasets.IMDB.splits(TEXT,LABEL)

vars(trainset[0])

downloading aclImdb_v1.tar.gz


100%|██████████| 84.1M/84.1M [00:11<00:00, 7.48MB/s]


{'label': 'pos',
 'text': ['this',
  'is',
  'one',
  'of',
  'the',
  'best',
  'movies',
  'out',
  'there',
  'and',
  "that's",
  'saying',
  'a',
  'lot',
  'being',
  'that',
  'it',
  'was',
  'for',
  'television.',
  'i',
  'really',
  'wish',
  'it',
  'was',
  'on',
  'd.v.d.<br',
  '/><br',
  '/>helen',
  'hunt',
  'gave',
  'such',
  'a',
  'raw',
  'performance.',
  'she',
  'played',
  'a',
  'rookie',
  'cop',
  'thrown',
  'into',
  'serial',
  'killer',
  'case',
  'perfectly.',
  'when',
  'she',
  'falls',
  'apart',
  'because',
  'he',
  'kills',
  'another',
  'kid',
  'it',
  'was',
  'amazing.',
  'she',
  'is',
  'so',
  'alone,',
  'so',
  'he',
  'gets',
  'to',
  'her.',
  'when',
  'she',
  'talks',
  'about',
  'her',
  'mother!',
  'wow!<br',
  '/><br',
  '/>steven',
  'weber',
  'as',
  'the',
  'serial',
  'killer',
  'was',
  'so',
  'shocking!',
  'he',
  'really',
  'brought',
  'her',
  'into',
  'his',
  'dark',
  'world.',
  'it',
  'was',
  'osc

In [6]:
from torchtext.vocab import GloVe

TEXT.build_vocab(trainset, vectors=GloVe(name='6B', dim=300), max_size=10000, min_freq=10)
LABEL.build_vocab(trainset)

embedding_layer = nn.Embedding.from_pretrained(TEXT.vocab.vectors, freeze=False)
embedding_layer(torch.LongTensor([10])) # nn.Embedding의 초기 입력값으로 TEXT.vocab.vectors 사용

.vector_cache/glove.6B.zip: 862MB [02:42, 5.30MB/s]                           
100%|█████████▉| 399999/400000 [00:47<00:00, 8420.33it/s]


tensor([[-2.0437e-01,  1.6431e-01,  4.1794e-02, -1.3708e-01, -2.9779e-01,
          3.3440e-01, -6.9955e-02, -6.8036e-02,  1.0604e-01, -2.0337e+00,
          1.7977e-01, -7.7403e-02, -1.9518e-01,  1.8324e-01,  3.0017e-02,
         -5.4762e-02, -4.5725e-01, -2.4509e-02,  5.7387e-02, -3.4878e-01,
          3.9696e-02,  4.4826e-01, -5.8462e-02,  4.1181e-01, -3.5411e-02,
         -1.4722e-01,  1.0740e-01, -2.5896e-01, -1.1658e-01,  1.9822e-01,
          3.2850e-01,  2.4177e-01, -5.7177e-01, -5.6442e-02, -9.6437e-01,
          3.4482e-01,  5.4639e-02,  2.3828e-01, -1.9139e-01,  3.0899e-01,
          2.8044e-01, -3.3814e-02, -2.5436e-01,  1.5373e-02,  1.6341e-01,
          2.6352e-01,  1.5812e-01,  3.2044e-01, -2.3082e-01,  2.6050e-01,
          2.0606e-01, -8.9769e-02, -1.0055e-01,  7.0378e-02, -2.7452e-02,
          2.7959e-01, -9.5862e-02,  2.0574e-01,  2.9522e-01, -1.2285e-02,
          1.1164e-01, -5.1373e-02,  4.6106e-01,  2.3014e-02, -3.7141e-01,
         -2.4166e-01,  3.3773e-02,  3.